This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/"))
print(os.listdir("../working/"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

['learn-together']
['__notebook_source__.ipynb', 'submission.csv', '.ipynb_checkpoints']
/kaggle/input/learn-together/sample_submission.csv
/kaggle/input/learn-together/test.csv
/kaggle/input/learn-together/train.csv


In [18]:
#load data, I had an issue with the data and hacked the data into the kernel
#dftrain=pd.read_csv('/kaggle/input/train.csv')
#dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [ ]:
#file shape
print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

In [ ]:
#features types
print(dftrain.dtypes)
print(dftest.dtypes)

In [ ]:
#validate files - nans per feature
print(dftrain.isnull().sum(axis = 0))
print(dftest.isnull().sum(axis = 0))

In [ ]:
#validate files - no rows with all zeros
print(dftrain[dftrain.drop(['Id','Cover_Type'], axis=1).eq(0).all(1)].empty)
print(dftest[dftest.drop('Id', axis=1).eq(0).all(1)].empty)

In [19]:
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

In [20]:
#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

In [21]:
#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173                      

In [ ]:
#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

In [22]:
# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

In [23]:
#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

In [24]:
#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [25]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

num_features = 54

model = Sequential()
model.add(Dense(units=num_features, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*10, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*10, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*10, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*10, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer='Adadelta',
              optimizer='Adam',
              metrics=['accuracy'])

In [26]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 4s 314us/sample - loss: 1.0548 - acc: 0.5422 - val_loss: 0.8657 - val_acc: 0.6329
Epoch 2/250
12096/12096 [==============================] - 3s 271us/sample - loss: 0.8762 - acc: 0.6323 - val_loss: 0.8305 - val_acc: 0.6594
Epoch 3/250
12096/12096 [==============================] - 3s 274us/sample - loss: 0.8029 - acc: 0.6648 - val_loss: 0.7871 - val_acc: 0.6587
Epoch 4/250
12096/12096 [==============================] - 3s 272us/sample - loss: 0.7697 - acc: 0.6792 - val_loss: 0.7076 - val_acc: 0.7004
Epoch 5/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.7486 - acc: 0.6822 - val_loss: 0.6858 - val_acc: 0.7199
Epoch 6/250
12096/12096 [==============================] - 3s 276us/sample - loss: 0.7262 - acc: 0.6939 - val_loss: 0.6559 - val_acc: 0.7189
Epoch 7/250
12096/12096 [==============================] - 3s 275us/sample - loss: 0.7132 - acc: 0.7051 -

12096/12096 [==============================] - 3s 267us/sample - loss: 0.5156 - acc: 0.7856 - val_loss: 0.5219 - val_acc: 0.7953
Epoch 59/250
12096/12096 [==============================] - 3s 270us/sample - loss: 0.5168 - acc: 0.7873 - val_loss: 0.5128 - val_acc: 0.8039
Epoch 60/250
12096/12096 [==============================] - 3s 268us/sample - loss: 0.5037 - acc: 0.7917 - val_loss: 0.5258 - val_acc: 0.7907
Epoch 61/250
12096/12096 [==============================] - 3s 266us/sample - loss: 0.5097 - acc: 0.7873 - val_loss: 0.5159 - val_acc: 0.7993
Epoch 62/250
12096/12096 [==============================] - 3s 266us/sample - loss: 0.5005 - acc: 0.7890 - val_loss: 0.5250 - val_acc: 0.7996
Epoch 63/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.5028 - acc: 0.7937 - val_loss: 0.5270 - val_acc: 0.7927
Epoch 64/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.4990 - acc: 0.7926 - val_loss: 0.5081 - val_acc: 0.8036
Epoch 65/250
12096/

12096/12096 [==============================] - 3s 266us/sample - loss: 0.4520 - acc: 0.8131 - val_loss: 0.4830 - val_acc: 0.8171
Epoch 116/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.4397 - acc: 0.8201 - val_loss: 0.5139 - val_acc: 0.8155
Epoch 117/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.4453 - acc: 0.8182 - val_loss: 0.5060 - val_acc: 0.8072
Epoch 118/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.4383 - acc: 0.8179 - val_loss: 0.4788 - val_acc: 0.8221
Epoch 119/250
12096/12096 [==============================] - 3s 266us/sample - loss: 0.4429 - acc: 0.8211 - val_loss: 0.4867 - val_acc: 0.8138
Epoch 120/250
12096/12096 [==============================] - 3s 266us/sample - loss: 0.4415 - acc: 0.8182 - val_loss: 0.5038 - val_acc: 0.8191
Epoch 121/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.4541 - acc: 0.8146 - val_loss: 0.5040 - val_acc: 0.8013
Epoch 122/250

12096/12096 [==============================] - 3s 275us/sample - loss: 0.4096 - acc: 0.8366 - val_loss: 0.4836 - val_acc: 0.8284
Epoch 173/250
12096/12096 [==============================] - 3s 278us/sample - loss: 0.4182 - acc: 0.8333 - val_loss: 0.5057 - val_acc: 0.8128
Epoch 174/250
12096/12096 [==============================] - 3s 271us/sample - loss: 0.4059 - acc: 0.8346 - val_loss: 0.4829 - val_acc: 0.8280
Epoch 175/250
12096/12096 [==============================] - 3s 277us/sample - loss: 0.4088 - acc: 0.8377 - val_loss: 0.5036 - val_acc: 0.8181
Epoch 176/250
12096/12096 [==============================] - 3s 272us/sample - loss: 0.4029 - acc: 0.8387 - val_loss: 0.4983 - val_acc: 0.8201
Epoch 177/250
12096/12096 [==============================] - 3s 274us/sample - loss: 0.4054 - acc: 0.8379 - val_loss: 0.4852 - val_acc: 0.8251
Epoch 178/250
12096/12096 [==============================] - 3s 281us/sample - loss: 0.4138 - acc: 0.8349 - val_loss: 0.4904 - val_acc: 0.8161
Epoch 179/250

12096/12096 [==============================] - 3s 267us/sample - loss: 0.3914 - acc: 0.8449 - val_loss: 0.4823 - val_acc: 0.8261
Epoch 230/250
12096/12096 [==============================] - 3s 269us/sample - loss: 0.3952 - acc: 0.8411 - val_loss: 0.4919 - val_acc: 0.8251
Epoch 231/250
12096/12096 [==============================] - 3s 272us/sample - loss: 0.3918 - acc: 0.8419 - val_loss: 0.4808 - val_acc: 0.8294
Epoch 232/250
12096/12096 [==============================] - 3s 274us/sample - loss: 0.4005 - acc: 0.8405 - val_loss: 0.5127 - val_acc: 0.8112
Epoch 233/250
12096/12096 [==============================] - 3s 270us/sample - loss: 0.4156 - acc: 0.8352 - val_loss: 0.5008 - val_acc: 0.8118
Epoch 234/250
12096/12096 [==============================] - 3s 272us/sample - loss: 0.4024 - acc: 0.8394 - val_loss: 0.4825 - val_acc: 0.8247
Epoch 235/250
12096/12096 [==============================] - 3s 272us/sample - loss: 0.3881 - acc: 0.8444 - val_loss: 0.4892 - val_acc: 0.8307
Epoch 236/250

In [27]:
# Predict!!
y_predict = model.predict(x_predict)

In [ ]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

In [28]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

In [29]:
#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv